In [ ]:
from openai import OpenAI
MODEL_NAME = "gpt-4o-mini" 

# OpenAIクライアントの初期化
# client = OpenAI(api_key="")
# メッセージを格納するリスト
SYSTEM_PROMPT = """
あなたは関西弁の頼れる先輩エンジニア『ミナト先輩』です。
- 一人称は「私」。相手は「あなた」or「君」。
- 口調はやさしくフレンドリー（例: 〜やで / 〜やね）。
- 役割: 初学者を励ましつつ、手順を短く箇条書きで示す。必要ならコード例も提示。
- 常に日本語で回答。絵文字は必要なときのみ1つまで。
- 不確かな点は推測せず、正直に伝える。
- どのような長さの対話になっても、このキャラクターを維持すること。
"""

messages = []
MAX_TURNS = 8  # ユーザー↔アシスタントの「往復」上限。system はカウントしない。

while True:
    # ユーザーからの質問を受付
    message = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if message.strip() == "":
        break
    print(f"質問:{message}")

    # メッセージにユーザーからの質問を追加（履歴は system を含めない）
    messages.append({"role": "user", "content": message.strip()})

    # 直近 MAX_TURNS 往復（= 最大 2*MAX_TURNS 発話）に履歴を制限
    trimmed_history = messages[-(MAX_TURNS * 2):]

    # API へ渡す最終メッセージ列：必ず system を先頭に付与
    request_messages = [{"role": "system", "content": SYSTEM_PROMPT}] + trimmed_history

    # APIへリクエスト（streaming）
    stream = client.chat.completions.create(
        model=MODEL_NAME,
        messages=request_messages,
        stream=True,
    )

    # 言語モデルからの回答を表示
    response_message = ""
    for chunk in stream:
        if chunk.choices:
            nxt = chunk.choices[0].delta.content
            if nxt is not None:
                response_message += nxt
                print(nxt, end='', flush=True)

    # メッセージに言語モデルからの回答を追加（履歴には system を保存しない）
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")


---ご利用ありがとうございました！---
